In [1]:
import pandas as pd
import numpy as np
import zipfile
import csv
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
!conda install -c conda-forge geocoder --yes
import geocoder 
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [2]:
!wget -q -O 'CEP-dados-2018-UTF8.zip' http://cep.la/CEP-dados-2018-UTF8.zip
print('Data downloaded!')

Data downloaded!


In [3]:
with zipfile.ZipFile("CEP-dados-2018-UTF8.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [4]:
df = pd.read_fwf('ceps.txt', header = None)
df.head()

,0,1,2,3,4,5,6
0,1001000,São,Paulo/SP,Sé\tPraça da Sé - lado ímpar,NaN,NaN,NaN
1,1001001,São,Paulo/SP,Sé\tPraça da Sé - lado par,NaN,NaN,NaN
2,1001010,São,Paulo/SP,Sé\tRua Filipe de Oliveira,NaN,NaN,NaN
3,1001900,São,Paulo/SP,"Sé\tPraça da Sé, 108 \t UNESP - Universidade E...",NaN,NaN,NaN
4,1001901,São,Paulo/SP,"Sé\tPraça da Sé, 371 \t Edifício Santa Lídia",NaN,NaN,NaN


In [5]:
df1 = df[1] + " " + df[2]
df2 = pd.DataFrame(df1, columns=["CS"])


df2 = df2["CS"].str.split('/', expand=True)
df2.columns = ['City', 'State']
df2['CEP'] = df[0]

df2.head()

,City,State,CEP
0,São Paulo,SP,1001000
1,São Paulo,SP,1001001
2,São Paulo,SP,1001010
3,São Paulo,SP,1001900
4,São Paulo,SP,1001901


In [6]:
df3 = pd.DataFrame(df[3])

df4 = df3[3].str.split('\t', expand=True)
df4['CEP'] = df[0]

df5 = df4.drop([2, 3], axis = 1)
df5.columns = ['Neighborhood', 'Street', 'CEP']

df5.head()

,Neighborhood,Street,CEP
0,Sé,Praça da Sé - lado ímpar,1001000
1,Sé,Praça da Sé - lado par,1001001
2,Sé,Rua Filipe de Oliveira,1001010
3,Sé,"Praça da Sé, 108",1001900
4,Sé,"Praça da Sé, 371",1001901


In [7]:
dfm = pd.merge(df2, df5, on='CEP')
dfm.head()

,City,State,CEP,Neighborhood,Street
0,São Paulo,SP,1001000,Sé,Praça da Sé - lado ímpar
1,São Paulo,SP,1001001,Sé,Praça da Sé - lado par
2,São Paulo,SP,1001010,Sé,Rua Filipe de Oliveira
3,São Paulo,SP,1001900,Sé,"Praça da Sé, 108"
4,São Paulo,SP,1001901,Sé,"Praça da Sé, 371"


In [8]:
dfm.tail()

,City,State,CEP,Neighborhood,Street
732758,São João Bos,None,99975970,o (Ciríaco)/RS - Distrito,Centro
732759,Cru altinha,None,99978000,Ciríaco)/RS - Distrito,None
732760,Dav d Canaba,None,99980000,ro/RS,None
732761,Dav d Canaba,None,99980970,ro/RS,Centro
732762,Dav d Canaba,None,99980974,ro/RS,Capela São José do Capingui (Distrito)


In [9]:
dt = dfm[dfm['City'] == 'São Paulo']
dt.tail()

,City,State,CEP,Neighborhood,Street
88115,São Paulo,SP,8490885,Jardim Pedra Branca,Rua Cambará-Branco
88116,São Paulo,SP,8490886,Jardim Pedra Branca,Rua Erva-Baleeira
88117,São Paulo,SP,8490890,Jardim Pedra Branca,Rua Buritizinho
88118,São Paulo,SP,8490895,Jardim Pedra Branca,Rua Rosa da Venezuela
88119,São Paulo,SP,8491030,Guaianazes,Rua Lauro


In [10]:
dt1 = dt.drop_duplicates('Neighborhood', keep='first')
dt1.shape

(1920, 5)

In [11]:
# Creating a function to get the Lat Long data from the Postal Code
def get_geocoder(postal_code_from_df):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, São Paulo, São Paulo'.format(str(postal_code_from_df).strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [12]:
# Adding Latitude and Longitude columns to dataframe
dt1['Latitude'], dt1['Longitude'] = zip(*dt1['Neighborhood'].apply(get_geocoder))
dt1.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,City,State,CEP,Neighborhood,Street,Latitude,Longitude
0,São Paulo,SP,1001000,Sé,Praça da Sé - lado ímpar,-23.55003,-46.63114
9,São Paulo,SP,1002020,Centro,Viaduto do Chá,-23.54869,-46.63303
173,São Paulo,SP,1017000,Brás,Avenida Rangel Pestana - até 499/500,-23.54277,-46.62000
337,São Paulo,SP,1035100,República,Avenida São João - de 651 a 1339 - lado ímpar,-23.54503,-46.64368
540,São Paulo,SP,1101000,Luz,Avenida Santos Dumont - até 999/1000,-23.52745,-46.62687


In [13]:
address = 'São Paulo, São Paulo'

geolocator = Nominatim(user_agent="sp_sp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('São Paulo, São Paulo Latitude and Logitude: {}, {}.'.format(latitude, longitude))

São Paulo, São Paulo Latitude and Logitude: -23.5506507, -46.6333824.


In [14]:
# create map of New York using latitude and longitude values
map1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng in zip(dt1['Latitude'], dt1['Longitude']):
    folium.CircleMarker([lat, lng],radius=1).add_to(map1)  
    
map1

In [15]:
LIMIT = 100

CLIENT_ID = 'IP4BV5KG2VGZXNWTDR4JAGV0H1CWPXOTOFIPCS5YFVJNE5TO' # your Foursquare ID
CLIENT_SECRET = 'R4D1BJYGYFLSH453ZDQ04VHLTK2LLBKUXXWE11TSDKDMFSOX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IP4BV5KG2VGZXNWTDR4JAGV0H1CWPXOTOFIPCS5YFVJNE5TO
CLIENT_SECRET:R4D1BJYGYFLSH453ZDQ04VHLTK2LLBKUXXWE11TSDKDMFSOX


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
Tven = getNearbyVenues(names = dt1['Neighborhood'],
                                   latitudes=dt1['Latitude'],
                                   longitudes=dt1['Longitude'],
                                  )

Sé
Centro
Brás
República
Luz
Ponte Pequena
Bom Retiro
Canindé
Barra Funda
Várzea da Barra Funda
Parque Industrial Tomas Edson
Santa Cecília
Perdizes
Campos Elíseos
Santa Efigênia
Vila Buarque
Higienópolis
Consolação
Pacaembu
Sumaré
Cerqueira César
Bela Vista
Liberdade
Morro dos Ingleses
Jardim Paulista
Jardim América
Jardim Paulistano
Jardins
Jardim Europa
Itaim Bibi
Pinheiros
Cambuci
Aclimação
Vila Deodoro
Jardim da Glória
Jardim Glória
Vila Monumento
Santana
Parque Anhembi
Vila Bianca
Carandiru
Vila Guilherme
Vila Matias
Jardim São Paulo(Zona Norte)
Vila Paulicéia
Parque Veloso
Vila Paiva
Vila Isolina Mazzei
Vila Leonor
Vila Mazzei
Vila Maria Baixa
Vila Maria
Vila Maria Alta
Jardim Japão
Vila Sabrina
Parque Novo Mundo
Jardim Guanca
Jardim Brasil (Zona Norte)
Jardim Julieta
Jardim Andaraí
Conjunto Promorar Vila Maria III
Parque Vila Maria
Promorar Vila Maria
Vila Gustavo
Vila Constança
Vila Ede
Vila Medeiros
Jardim Neila
Vila São Camilo
Vila Nivi
Parque Edu Chaves
Jardim Modelo
Jaçanã

Jardim Iara
Jardim São Francisco (Zona Leste)
Parque Cruzeiro do Sul (Vila Formosa)
Capão do Embira
Parque Santo Eduardo
Parque Santo Antônio (Aricanduva)
Jardim Galli
Jardim Colorado
Vila Carrão
Vila Nova Manchester
Vila Antonina
Jardim Textil
Vila Mafra
Vila Santa Isabel
Vila Fernandes
Parque Maria Luiza
Jardim Aricanduva
Jardim Vila Formosa
Vila Antonieta
Jardim Record
Jardim Cotching
Jardim Piqueroby
Vila Dona Sara
Jardim Haia do Carrão
Vila Nova York
Vila Aricanduva
Vila Carlos de Campos
Aricanduva
Chácara Seis de Outubro
Vila Matilde
Vila Talarico
Vila Euthalia
Vila Dalila
Jardim Maringá
Jardim Fernandes
Jardim Santa Terezinha (Zona Leste)
Vila Nova Savoia
Cidade Patriarca
Vila Guilhermina
Jardim Assunção
Jardim Triana
Jardim Hercilia
Vila Nhocune
Jardim São José (Artur Alvim)
Jardim Santo Antônio
Vila Santa Teresa (Zona Leste)
Artur Alvim
Parque Artur Alvim
Parque Savoy City
Jardim Arize
Jardim Santa Maria
Jardim Eliane
Jardim Itapema
Jardim Marília
Jardim Brasília (Zona Leste)


Praia Paulistinha
Jardim Edilene
Jardim São Benedito
Jardim São Judas Tadeu
Jardim São Vicente
Jardim Guanhembu
Jardim Maria Rita
Jardim Rio Bonito
Jardim Marcel
Jardim Quarto Centenário
Jardim Kika
Jardim Mália II
Jardim dos Bichinhos
Jardim Bichinhos
Parque América
Jardim Mália I
Parque Alto do Rio Bonito
Jardim Imperial
Vila Rubi
Jardim Reimberg
Parque Santana
Jardim Nizia
Jardim do Alto
Parque das Árvores
Rio Bonito
Jardim Bela Vista (Zona Sul)
Jardim Represa
Jardim das Praias
Jardim Sertãozinho
Vila Represa
Jardim Real
Jardim Cliper
Jardim Esmeralda (Zona Sul)
Jardim Iporanga
Jardim Bonito
Jardim das Imbuias
Jardim das Camélias (Zona Sul)
Jordanópolis
Vila Progresso (Zona Sul)
Jardim Presidente
Jardim Santa Rita
Jardim Pouso Alegre
Jardim Kioto
Parque Santa Cecília
Vila São José (Cidade Dutra)
Vila São José
Jardim Icaraí
Jardim Santa Edwiges (Grajaú)
Jardim Beatriz
Jardim Angelina
Jardim Floresta
Jardim Alpino
Jardim Panorama (Zona Sul)
Jardim Orbam
Vila Quintana
Conjunto Habitaci

Cidade dos Bandeirantes
Vila Albano
Jardim Guarau
Educandário
Jardim Luísa
Jardim Lúcia
Jardim Cláudia
Vila Borges
Jardim Batalha
Jardim Maria Augusta
Jardim Monte Belo (Raposo Tavares)
Raposo Tavares
Jardim Rubio
Jardim das Esmeraldas
Jardim Lúcio de Castro
Jardim Gilda Maria
Jardim Raposo Tavares
Jardim Cambara
Jardim Arpoador
Jardim Uirapuru
Jardim São Jorge (Raposo Tavares)
Jardim João XXIII
Jardim Paulo VI
Jardim Amaralina
Vila Nova Esperança
Rua Paraíso, 08 - Vila Nova Esperança - Butantã/SP. 
Parque Ipê
Vila Pirajussara
Jardim Boa Vista (Zona Oeste)
Jardim Pirajussara
Vila Indiana
Jardim Rizzo
Jardim Bonfiglioli
Jardim Everest
Jardim Guedala
Morumbi
Cidade Jardim
Vila Progredior
Jardim Leonor
Jardim Namba
Vila Inah
Vila Morse
Jardim Morumbi
Jardim Colombo
Vila Suzana
Jardim Londrina
Jardim Monte Kemel
Jardim Santa Filomena
Lar São Paulo
Parque Bairro Morumbi
Jardim Caboré
Jardim Vazani
Vila Andrade
Jardim Viana
Jardim Vitória Régia
Paraisópolis
Paraíso do Morumbi
Jardim Panorama

Jardim Guaianazes
Jardim Nova Guaianazes
Parque Guaianazes
Jardim do Campo
Vila Minerva
Vila Chabilândia
Núcleo Lageado
Vila Andes
Vila Fukuya
Lajeado
Jardim Fanganiello
Vila Iolanda(Lajeado)
Jardim Augusta
Vila Independente
Jardim Lourdes
Vila ABC
Cidade Popular
Jardins Recanto das Rosas
Jardim do Divino
Conjunto Habitacional Juscelino Kubitschek
Vila Roseira II
Cidade Tiradentes
Conjunto Habitacional Sitio Conceição
Jardim dos Pereiras
Jardim Santa Etelvina
Conjunto Habitacional Inácio Monteiro
Jardim Wilma Flor
Vila Iolanda II
Jardim São Raimundo
Conjunto Habitacional Barro Branco II
Jardim Vitória
Jardim Pérola II
Jardim Pérola III
Conjunto Habitacional Castro Alves
Conjunto Habitacional Barro Branco I
Jardim Pérola I
Conjunto Habitacional Santa Etelvina II
Conjunto Habitacional Santa Etelvina III
Palanque
Vila Hortência
Chácara Santa Etelvina
Santa Etelvina
Vila Paulista I
Vila Paulista II


In [18]:
Tven.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sé,-23.55003,-46.63114,Caixa Cultural,-23.549381,-46.632849,Art Gallery
1,Sé,-23.55003,-46.63114,Piero Pasta & Café,-23.548734,-46.632186,Italian Restaurant
2,Sé,-23.55003,-46.63114,Casa da Imagem,-23.548470,-46.632457,Art Museum
3,Sé,-23.55003,-46.63114,Solar da Marquesa de Santos,-23.548663,-46.632399,Historic Site
4,Sé,-23.55003,-46.63114,Pateo do Collegio,-23.547883,-46.633079,Historic Site
5,Sé,-23.55003,-46.63114,Cama & Café,-23.549264,-46.632285,Restaurant
6,Sé,-23.55003,-46.63114,Restaurante Sesc Carmo,-23.551733,-46.630162,American Restaurant
7,Sé,-23.55003,-46.63114,Casa de Francisca,-23.548733,-46.634763,Music Venue
8,Sé,-23.55003,-46.63114,Beco do Pinto,-23.548461,-46.632397,Historic Site
9,Sé,-23.55003,-46.63114,Café do Páteo,-23.547905,-46.632732,Café


#### Data Presentation

In [19]:
t1 = Tven.groupby('Venue Category').size().reset_index(name='counts')
#t1['counts'][1]
t1.sort_values(by=['counts'], inplace=True, ascending=False)


In [20]:
t1.head(10)

,Venue Category,counts
34,Bakery,2157
325,Pizza Place,1971
58,Brazilian Restaurant,1655
36,Bar,1608
187,Gym / Fitness Center,1068
348,Restaurant,1057
317,Pharmacy,1029
186,Gym,896
65,Burger Joint,857
184,Grocery Store,769


In [21]:
t2 = Tven.groupby(['Neighborhood', 'Venue Category']).size().reset_index(name='counts')
t2.sort_values(by=['counts'], inplace=True, ascending=False) 
t2.head(10)

,Neighborhood,Venue Category,counts
15043,Liberdade,Japanese Restaurant,18
19000,Usina Piratininga,Bar,14
8740,Jardim Mar Lune,Theater,13
224,Baronesa,Brazilian Restaurant,13
2693,Consolação,Bar,12
21342,Vila Dona Augusta,Italian Restaurant,11
917,Casa Grande,Brazilian Restaurant,11
18997,Usina Piratininga,Art Gallery,11
14514,Jardim do Divino,Restaurant,11
12204,Jardim Suzana,Bar,10


In [22]:
t3 = t2.groupby(['Neighborhood', 'Venue Category']).sum().reset_index()

In [23]:
t3.sort_values(['Neighborhood','counts'],ascending=False).groupby('Neighborhood').head(3)

,Neighborhood,Venue Category,counts
27068,Área Rural de São Paulo,Hotel,7
27051,Área Rural de São Paulo,Brazilian Restaurant,6
27056,Área Rural de São Paulo,Coffee Shop,5
27028,Água Rasa,Brazilian Restaurant,3
27027,Água Rasa,Bakery,2
27035,Água Rasa,Farmers Market,2
27019,Água Funda,Brazilian Restaurant,1
27020,Água Funda,Café,1
27021,Água Funda,Furniture / Home Store,1
27007,Água Fria,Japanese Restaurant,6


#### Continuing Analysis

In [45]:
a = Tven.groupby('Neighborhood').count().reset_index()
b = a[['Neighborhood','Venue']]
b.columns = ['Neighborhood','counts']
b.sort_values(by=['counts'], inplace=True, ascending=False) 
b.head(10)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,counts
1871,Área Rural de São Paulo,100
1071,Moema,100
1278,Santa Amélia,100
1266,Roda a Roda Jequiti,100
1258,República,100
1245,Protendit,100
1232,Pinheiros,100
1180,Parque Rodrigues Alves,100
244,Jardim Aladim,100
321,Jardim Bronzato,100


In [26]:
print('There are a total of {} uniques venues categories.'.format(len(Tven['Venue Category'].unique())))

There are a total of 457 uniques venues categories.


In [27]:

Tven1 = pd.get_dummies(Tven[['Venue Category']], prefix="", prefix_sep="")
Tven1['Neighborhood'] = Tven['Neighborhood'] 
fixed_columns = [Tven1.columns[-1]] + list(Tven1.columns[:-1])
Tven1 = Tven1[fixed_columns]

Tven1.head()

,Zoo Exhibit,Acai House,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,...,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
Tven1.shape


(37150, 457)

In [29]:
TvenGr = Tven1.groupby('Neighborhood').mean().reset_index()
TvenGr.head()

,Neighborhood,Zoo Exhibit,Acai House,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,...,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Aclimação,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alto da Boa Vista,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alto da Lapa,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alto da Mooca,0.0,0.018519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alto da Riviera,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
top5 = 5

for hood in TvenGr['Neighborhood']:
    print("----"+hood+"----")
    temp = TvenGr[TvenGr['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top5))
    print('\n')

----Aclimação----
                  venue  freq
0  Gym / Fitness Center  0.10
1     Korean Restaurant  0.08
2           Pizza Place  0.07
3                Bakery  0.07
4              Pharmacy  0.05


----Alto da Boa Vista----
                   venue  freq
0   Brazilian Restaurant  0.50
1    Japanese Restaurant  0.17
2            Salad Place  0.17
3      Food & Drink Shop  0.17
4  Outdoors & Recreation  0.00


----Alto da Lapa----
               venue  freq
0        Pizza Place  0.16
1       Soccer Field  0.11
2         Restaurant  0.11
3              Plaza  0.05
4  Health Food Store  0.05


----Alto da Mooca----
              venue  freq
0               Bar  0.13
1              Café  0.06
2      Burger Joint  0.06
3  Sushi Restaurant  0.06
4               Spa  0.04


----Alto da Riviera----
           venue  freq
0            Gym   0.2
1          Diner   0.2
2  Hot Dog Joint   0.2
3    Bus Station   0.2
4    Pizza Place   0.2


----Alto de Pinheiros----
               venue  freq
0  C

            venue  freq
0          Bakery  0.22
1     Pizza Place  0.17
2             Gym  0.11
3           Plaza  0.06
4  Clothing Store  0.06


----Capão do Embira----
           venue  freq
0        Brewery   0.1
1  Grocery Store   0.1
2         Bakery   0.1
3       Pharmacy   0.1
4   Soccer Field   0.1


----Carandiru----
                    venue  freq
0     Arts & Crafts Store  0.06
1                  Bakery  0.06
2       Convenience Store  0.06
3  Furniture / Home Store  0.06
4    Brazilian Restaurant  0.06


----Casa Grande----
                  venue  freq
0  Brazilian Restaurant  0.15
1                  Café  0.07
2           Pizza Place  0.05
3                Bakery  0.04
4              Tea Room  0.04


----Casa Verde----
                  venue  freq
0  Gym / Fitness Center  0.11
1            Restaurant  0.08
2                   Bar  0.08
3                   Gym  0.05
4            Food Truck  0.03


----Casa Verde Alta----
               venue  freq
0      Grocery Store  0.

           venue  freq
0  Movie Theater  0.12
1            Gym  0.12
2          Diner  0.12
3     Hookah Bar  0.12
4         Bakery  0.12


----Chácara Seis de Outubro----
                  venue  freq
0  Brazilian Restaurant  0.12
1           Pizza Place  0.12
2      Department Store  0.06
3     Convenience Store  0.06
4            Food Court  0.06


----Chácara São João----
                   venue  freq
0            Wings Joint   0.5
1      Convenience Store   0.5
2            Zoo Exhibit   0.0
3  Outdoors & Recreation   0.0
4     Persian Restaurant   0.0


----Chácara São Luís----
                       venue  freq
0                        Bar  0.14
1                     Bakery  0.07
2  Middle Eastern Restaurant  0.07
3                      Plaza  0.07
4         Chinese Restaurant  0.07


----Chácara São Silvestre----
          venue  freq
0          Farm   0.5
1          Food   0.5
2   Zoo Exhibit   0.0
3  Outlet Store   0.0
4  Perfume Shop   0.0


----Chácara Tatuapé----
        

                  venue  freq
0                   Gym  0.08
1              Pharmacy  0.08
2  Fast Food Restaurant  0.08
3                  Café  0.08
4         Grocery Store  0.08


----Cipó do Meio----
                     venue  freq
0                      Gym  0.38
1  Comfort Food Restaurant  0.12
2             Dessert Shop  0.12
3              Pizza Place  0.12
4   Furniture / Home Store  0.12


----City América----
                venue  freq
0            Pharmacy  0.15
1    Sushi Restaurant  0.12
2         Pizza Place  0.08
3  Chinese Restaurant  0.08
4         Supermarket  0.04


----Cohab Monet----
                  venue  freq
0              Pharmacy  0.08
1           Supermarket  0.06
2            Restaurant  0.06
3  Brazilian Restaurant  0.04
4           Pizza Place  0.04


----Colônia (Zona Leste)----
                venue  freq
0           Nightclub   0.5
1                 Gym   0.5
2        Outlet Store   0.0
3  Persian Restaurant   0.0
4        Perfume Shop   0.0


----C

                venue  freq
0         Snack Place  0.25
1  Athletics & Sports  0.25
2                 Bar  0.25
3              Market  0.25
4         Zoo Exhibit  0.00


----Conjunto Promorar São Luis----
                  venue  freq
0              Pharmacy  0.18
1         Grocery Store  0.18
2  Brazilian Restaurant  0.09
3        Ice Cream Shop  0.09
4           Flower Shop  0.09


----Conjunto Promorar Vila Maria III----
             venue  freq
0        Cafeteria  0.14
1        Rest Area  0.14
2  Piercing Parlor  0.14
3         Bus Line  0.14
4      Pizza Place  0.14


----Conjunto Residencial Alpes do Jaraguá----
                       venue  freq
0               Soccer Field  0.23
1                 Food Truck  0.15
2  Middle Eastern Restaurant  0.08
3                Pizza Place  0.08
4                Supermarket  0.08


----Conjunto Residencial Bandeirantes----
                  venue  freq
0        Farmers Market  0.25
1          Optical Shop  0.25
2            Food Truck  0.25


...

                     venue  freq
0              Pizza Place  0.14
1                   Bakery  0.14
2     Brazilian Restaurant  0.07
3  Fruit & Vegetable Store  0.07
4           Gymnastics Gym  0.07


----Vila Nova Cachoeirinha----
            venue  freq
0     Flea Market  0.25
1  Sandwich Place  0.25
2             Gym  0.25
3     Pizza Place  0.25
4     Zoo Exhibit  0.00


----Vila Nova Caledônia----
                    venue  freq
0                  Bakery  0.38
1          Farmers Market  0.12
2  Furniture / Home Store  0.12
3               Pet Store  0.12
4          Sandwich Place  0.12


----Vila Nova Carolina----
             venue  freq
0      Coffee Shop   0.1
1             Café   0.1
2  Thai Restaurant   0.1
3  Soba Restaurant   0.1
4              Gym   0.1


----Vila Nova Conceição----
                  venue  freq
0    Italian Restaurant  0.07
1            Restaurant  0.06
2                  Café  0.05
3  Brazilian Restaurant  0.05
4        Chocolate Shop  0.04


----Vila Nov

                  venue  freq
0           Pizza Place  0.13
1                Bakery  0.09
2                   Gym  0.09
3  Gym / Fitness Center  0.09
4            Restaurant  0.09


----Vila Piauí----
         venue  freq
0  Pizza Place  0.33
1    BBQ Joint  0.17
2       Bakery  0.17
3   Pastelaria  0.17
4   Food Truck  0.17


----Vila Picinin----
                      venue  freq
0            Breakfast Spot  0.08
1                  Gym Pool  0.08
2                 BBQ Joint  0.08
3  Mediterranean Restaurant  0.08
4        Athletics & Sports  0.08


----Vila Pierina----
            venue  freq
0     Pizza Place  0.20
1             Bar  0.16
2    Soccer Field  0.08
3  Ice Cream Shop  0.08
4      Hookah Bar  0.04


----Vila Piracicaba----
                  venue  freq
0  Brazilian Restaurant  0.09
1  Gym / Fitness Center  0.07
2            Restaurant  0.07
3            Food Truck  0.07
4           Pizza Place  0.04


----Vila Pirajussara----
                  venue  freq
0           Pizz

                  venue  freq
0                Bakery   0.2
1                   Bar   0.2
2  Other Great Outdoors   0.2
3                Market   0.2
4   Housing Development   0.2


----Vila Rosa Molla----
                  venue  freq
0                Bakery   0.2
1                   Bar   0.2
2  Other Great Outdoors   0.2
3                Market   0.2
4   Housing Development   0.2


----Vila Rosaria----
                venue  freq
0       Grocery Store  0.22
1           BBQ Joint  0.11
2            Gym Pool  0.11
3   Convenience Store  0.11
4  Athletics & Sports  0.11


----Vila Roschel----
                venue  freq
0              Market   1.0
1         Zoo Exhibit   0.0
2        Outlet Store   0.0
3  Persian Restaurant   0.0
4        Perfume Shop   0.0


----Vila Rosina----
                   venue  freq
0    Rental Car Location   0.5
1              Pet Store   0.5
2            Zoo Exhibit   0.0
3   Outdoor Supply Store   0.0
4  Performing Arts Venue   0.0


----Vila Rubi----
    

                 venue  freq
0               Bakery  0.11
1                  Bar  0.08
2  Japanese Restaurant  0.08
3    Indian Sweet Shop  0.06
4             Pharmacy  0.06


----Vila Socorro----
                  venue  freq
0        Clothing Store  0.23
1  Brazilian Restaurant  0.09
2  Fast Food Restaurant  0.09
3                  Café  0.09
4      Department Store  0.05


----Vila Sofia----
          venue  freq
0  Dessert Shop  0.09
1  Burger Joint  0.09
2   Pizza Place  0.07
3        Bakery  0.07
4           Bar  0.04


----Vila Souza----
            venue  freq
0     Pizza Place  0.14
1       Pet Store  0.09
2  Farmers Market  0.09
3      Board Shop  0.05
4          Bakery  0.05


----Vila Stela----
                venue  freq
0     Harbor / Marina   0.4
1           Nightclub   0.2
2  Athletics & Sports   0.2
3         Snack Place   0.2
4  Persian Restaurant   0.0


----Vila Suiça----
                 venue  freq
0               Bakery  0.16
1  Japanese Restaurant  0.12
2   Misc

                   venue  freq
0          Women's Store  0.33
1                    Bar  0.33
2           Concert Hall  0.33
3            Zoo Exhibit  0.00
4  Outdoors & Recreation  0.00


----Vila Vera----
                       venue  freq
0                Pizza Place  0.19
1                        Gym  0.08
2                  Pet Store  0.08
3                   Pharmacy  0.08
4  Middle Eastern Restaurant  0.04


----Vila Verde----
                  venue  freq
0                Bakery  0.33
1  Brazilian Restaurant  0.17
2  Fast Food Restaurant  0.17
3          Soccer Field  0.17
4  Gym / Fitness Center  0.17


----Vila Vermelha----
               venue  freq
0        Pizza Place  0.16
1                Gym  0.11
2  Electronics Store  0.05
3         Baby Store  0.05
4              Motel  0.05


----Vila Vessoni----
                 venue  freq
0               Bakery  0.17
1        Grocery Store  0.08
2  Japanese Restaurant  0.08
3          Pizza Place  0.08
4                  Gym  0.08


In [31]:
def return_most_common_venues(row, top10):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top10]


top10 = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(top10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

TvenSr = pd.DataFrame(columns=columns)
TvenSr['Neighborhood'] = TvenGr['Neighborhood']

for ind in np.arange(TvenGr.shape[0]):
    TvenSr.iloc[ind, 1:] = return_most_common_venues(TvenGr.iloc[ind, :], top10)

TvenSr_copy = TvenSr.copy()
    
TvenSr.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aclimação,Gym / Fitness Center,Korean Restaurant,Pizza Place,Bakery,Pharmacy,Dessert Shop,Chinese Restaurant,Farmers Market,Sandwich Place,Pet Store
1,Alto da Boa Vista,Brazilian Restaurant,Salad Place,Japanese Restaurant,Food & Drink Shop,Fish Market,Flea Market,Fishing Store,Fishing Spot,Fish Taverna,Zoo
2,Alto da Lapa,Pizza Place,Soccer Field,Restaurant,Japanese Restaurant,Park,Tennis Court,Health Food Store,Beer Store,Gym / Fitness Center,Movie Theater
3,Alto da Mooca,Bar,Burger Joint,Café,Sushi Restaurant,Dance Studio,Spa,Japanese Restaurant,Dessert Shop,Pharmacy,Deli / Bodega
4,Alto da Riviera,Pizza Place,Diner,Gym,Hot Dog Joint,Bus Station,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop


In [147]:
TvenSr = TvenSr_copy.copy()
k = 10

TvenGrCluster = TvenGr.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(TvenGrCluster)
kmeans.labels_[0:10]
TvenSr.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aclimação,Gym / Fitness Center,Korean Restaurant,Pizza Place,Bakery,Pharmacy,Dessert Shop,Chinese Restaurant,Farmers Market,Sandwich Place,Pet Store
1,Alto da Boa Vista,Brazilian Restaurant,Salad Place,Japanese Restaurant,Food & Drink Shop,Fish Market,Flea Market,Fishing Store,Fishing Spot,Fish Taverna,Zoo
2,Alto da Lapa,Pizza Place,Soccer Field,Restaurant,Japanese Restaurant,Park,Tennis Court,Health Food Store,Beer Store,Gym / Fitness Center,Movie Theater
3,Alto da Mooca,Bar,Burger Joint,Café,Sushi Restaurant,Dance Studio,Spa,Japanese Restaurant,Dessert Shop,Pharmacy,Deli / Bodega
4,Alto da Riviera,Pizza Place,Diner,Gym,Hot Dog Joint,Bus Station,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop


In [148]:
TvenSr.insert(0, 'Cluster Labels', kmeans.labels_)
TvenSr.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5,Aclimação,Gym / Fitness Center,Korean Restaurant,Pizza Place,Bakery,Pharmacy,Dessert Shop,Chinese Restaurant,Farmers Market,Sandwich Place,Pet Store
1,7,Alto da Boa Vista,Brazilian Restaurant,Salad Place,Japanese Restaurant,Food & Drink Shop,Fish Market,Flea Market,Fishing Store,Fishing Spot,Fish Taverna,Zoo
2,0,Alto da Lapa,Pizza Place,Soccer Field,Restaurant,Japanese Restaurant,Park,Tennis Court,Health Food Store,Beer Store,Gym / Fitness Center,Movie Theater
3,7,Alto da Mooca,Bar,Burger Joint,Café,Sushi Restaurant,Dance Studio,Spa,Japanese Restaurant,Dessert Shop,Pharmacy,Deli / Bodega
4,2,Alto da Riviera,Pizza Place,Diner,Gym,Hot Dog Joint,Bus Station,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop


In [149]:
Tmerge = dt1
Tmerge = Tmerge.join(TvenSr.set_index('Neighborhood'), on='Neighborhood')
Tmerge.head()

,City,State,CEP,Neighborhood,Street,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,São Paulo,SP,1001000,Sé,Praça da Sé - lado ímpar,-23.55003,-46.63114,5.0,Bookstore,Bakery,Jewelry Store,Historic Site,Snack Place,Miscellaneous Shop,Café,Brazilian Restaurant,Music Store,Arts & Crafts Store
9,São Paulo,SP,1002020,Centro,Viaduto do Chá,-23.54869,-46.63303,5.0,Café,Brazilian Restaurant,Bookstore,Jewelry Store,Chocolate Shop,Bar,Bakery,Snack Place,Historic Site,Coffee Shop
173,São Paulo,SP,1017000,Brás,Avenida Rangel Pestana - até 499/500,-23.54277,-46.62000,5.0,Brazilian Restaurant,Clothing Store,Furniture / Home Store,Italian Restaurant,Shopping Mall,Hotel,Department Store,Burger Joint,Restaurant,Shoe Store
337,São Paulo,SP,1035100,República,Avenida São João - de 651 a 1339 - lado ímpar,-23.54503,-46.64368,7.0,Coffee Shop,Brazilian Restaurant,Bar,Italian Restaurant,Theater,Restaurant,Café,Hotel,Cocktail Bar,Vegetarian / Vegan Restaurant
540,São Paulo,SP,1101000,Luz,Avenida Santos Dumont - até 999/1000,-23.52745,-46.62687,5.0,Brazilian Restaurant,Hotel,Noodle House,Bus Stop,Café,Snack Place,Mineiro Restaurant,Gym,Bakery,Farmers Market


In [150]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Tmerge1 = Tmerge.dropna()

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tmerge1['Latitude'], Tmerge1['Longitude'], Tmerge1['Neighborhood'], Tmerge1['Cluster Labels']):
    #label = folium.Popup(str(poi) + ' Cluster ' + str(Tmerge1['Cluster Labels']), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1.5,
        #popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)]).add_to(map_clusters)
       
map_clusters

In [151]:
# Cluster 0
c0 = Tmerge.loc[Tmerge['Cluster Labels'] == 0, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c00 = c0[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c00.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3277,0.0,Vila Bianca,Market,Pizza Place,Bar,Gym / Fitness Center,Gift Shop,Food & Drink Shop,Pharmacy,Martial Arts Dojo,Farmers Market,Bakery
3384,0.0,Jardim São Paulo(Zona Norte),Pizza Place,Japanese Restaurant,Bakery,Clothing Store,Pharmacy,Brazilian Restaurant,Spa,Café,Tattoo Parlor,Grocery Store
4251,0.0,Jardim Guanca,Restaurant,Pizza Place,Bakery,Gym / Fitness Center,Burger Joint,BBQ Joint,Hot Dog Joint,Pedestrian Plaza,Fireworks Store,Fishing Store
4615,0.0,Vila Medeiros,Pizza Place,Brazilian Restaurant,Ice Cream Shop,Market,Asian Restaurant,Grocery Store,Bakery,Pet Store,Food,Convenience Store
5968,0.0,Jardim Guapira,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Bakery,Other Great Outdoors,Grocery Store,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
6238,0.0,Furnas,Pizza Place,Fast Food Restaurant,Supermarket,Convenience Store,Food,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market
6581,0.0,Jardim Jaçanã,Fast Food Restaurant,Bakery,Ice Cream Shop,Supermarket,Gym / Fitness Center,Italian Restaurant,Pizza Place,Fireworks Store,Fish & Chips Shop,Floating Market
6735,0.0,Vila Virginia,Farmers Market,Pizza Place,Bakery,Candy Store,Sandwich Place,Pharmacy,Pet Store,Furniture / Home Store,Gym / Fitness Center,Diner
6937,0.0,Jardim Palmares (Zona Norte),Pizza Place,Bar,Juice Bar,Photography Studio,Food Truck,Food Court,Bistro,Food,Supermarket,Beer Garden
7454,0.0,Associação Sobradinho,Pizza Place,Bakery,Pet Store,Farmers Market,Furniture / Home Store,Metro Station,Gym / Fitness Center,Sandwich Place,Liquor Store,Pharmacy


In [152]:
c00.shape

(238, 12)

In [153]:
# Cluster 1
c1 = Tmerge.loc[Tmerge['Cluster Labels'] == 1, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c11 = c1[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c11.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12662,1.0,Jardim Marilu,Home Service,Athletics & Sports,Zoo,Farm,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market
36614,1.0,Parelheiros,Athletics & Sports,Zoo,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna
37322,1.0,Vila Isabel (Parelheiros),Athletics & Sports,Zoo,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna
44862,1.0,Jardim Everest,Athletics & Sports,Restaurant,Plaza,Zoo,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
85657,1.0,Terceira Divisão,Athletics & Sports,Zoo,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna


In [154]:
c11.shape

(5, 12)

In [156]:
# Cluster 2
c2 = Tmerge.loc[Tmerge['Cluster Labels'] == 2, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c22 = c2[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c22.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3730,2.0,Vila Leonor,Gym,Deli / Bodega,Pizza Place,Japanese Restaurant,Nightclub,Pharmacy,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store
6597,2.0,Parque Casa de Pedra,Toy / Game Store,Construction & Landscaping,Gym,Floating Market,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
7011,2.0,Mandaqui,Gym,Department Store,Farmers Market,Shoe Store,Gym / Fitness Center,Bar,Market,Pet Store,Pharmacy,Beer Bar
7016,2.0,Barro Branco (Zona Norte),Pharmacy,Pizza Place,Arts & Crafts Store,Gym,Movie Theater,Rest Area,Plaza,Fish & Chips Shop,Fish Taverna,Fish Market
8202,2.0,Vila Nova Cachoeirinha,Pizza Place,Gym,Sandwich Place,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store
8455,2.0,Chora Menino,Gym,Café,Board Shop,Grocery Store,Bike Rental / Bike Share,Gift Shop,Piano Bar,Bus Station,Athletics & Sports,Farmers Market
9166,2.0,Sítio do Morro,Supermarket,Bar,Tennis Court,Pet Store,Gym,Pizza Place,Zoo,Flea Market,Field,Financial or Legal Service
9209,2.0,Vila Diva (Zona Norte),Gym,Steakhouse,Convenience Store,Brazilian Restaurant,Bakery,Pharmacy,Grocery Store,Spa,Farmers Market,Food Truck
9410,2.0,Conjunto Habitacional Vila Nova Cachoeirinha,Pizza Place,Gym,Sandwich Place,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store
9791,2.0,Vila Dionisia,Farmers Market,Brazilian Restaurant,Supermarket,Gym,Sandwich Place,Floating Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store


In [158]:
c22.shape

(83, 12)

In [160]:
# Cluster 3
c3 = Tmerge.loc[Tmerge['Cluster Labels'] == 3, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c33 = c3[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c33.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3041,3.0,Vila Monumento,Supermarket,Bakery,Gym,Strip Club,Electronics Store,Mobile Phone Shop,Pharmacy,Burger Joint,Soccer Field,Museum
3433,3.0,Vila Paulicéia,Pharmacy,Bakery,BBQ Joint,Athletics & Sports,Big Box Store,Gym,Smoke Shop,Liquor Store,Dessert Shop,Fishing Store
3953,3.0,Vila Maria Alta,Bakery,Brazilian Restaurant,Gym / Fitness Center,Pizza Place,Ice Cream Shop,Metro Station,Health & Beauty Service,Market,Asian Restaurant,Pharmacy
4161,3.0,Vila Sabrina,Bakery,Pharmacy,Restaurant,Gym / Fitness Center,Gym,Soccer Field,Candy Store,Supermarket,Tea Room,Baby Store
4212,3.0,Parque Novo Mundo,Brazilian Restaurant,Bakery,Pharmacy,Coffee Shop,Park,Gym,Restaurant,Supermarket,Food & Drink Shop,Farmers Market
4360,3.0,Parque Vila Maria,Bakery,Health & Beauty Service,Veterinarian,Burger Joint,Spa,Bus Stop,Snack Place,Grocery Store,Skate Park,Fast Food Restaurant
4561,3.0,Vila Gustavo,Bakery,Brazilian Restaurant,Cosmetics Shop,Sandwich Place,Snack Place,Road,Restaurant,Pizza Place,Pharmacy,Market
4631,3.0,Jardim Neila,Bakery,Gym,Pharmacy,Arts & Crafts Store,Tea Room,Sandwich Place,Dumpling Restaurant,Zoo,Farmers Market,Field
4717,3.0,Vila Nivi,Bakery,Japanese Restaurant,Restaurant,Snack Place,Grocery Store,Bar,Gym / Fitness Center,Thrift / Vintage Store,Clothing Store,Sandwich Place
4788,3.0,Parque Edu Chaves,Gym / Fitness Center,Bakery,Northeastern Brazilian Restaurant,Soccer Field,Ice Cream Shop,Sandwich Place,Grocery Store,Trail,Gym,Plaza


In [162]:
c33.shape

(327, 12)

In [164]:
# Cluster 4
c4 = Tmerge.loc[Tmerge['Cluster Labels'] == 4, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c44 = c4[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c44.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4347,4.0,Conjunto Promorar Vila Maria III,Pizza Place,Piercing Parlor,Grocery Store,Bus Line,Rest Area,Jewelry Store,Cafeteria,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
4564,4.0,Vila Constança,Brazilian Restaurant,Hobby Shop,Dessert Shop,Gym,Sushi Restaurant,Grocery Store,Floating Market,Field,Financial or Legal Service,Fireworks Store
6417,4.0,Vila Cachoeira,Brazilian Restaurant,Grocery Store,Pizza Place,Men's Store,Paper / Office Supplies Store,Bakery,Snack Place,Fast Food Restaurant,Café,Fishing Spot
7249,4.0,Jardim das Rosas (Zona Norte),Brazilian Restaurant,Athletics & Sports,Pizza Place,Grocery Store,Snack Place,Restaurant,Gym / Fitness Center,Market,Fish & Chips Shop,Flea Market
7438,4.0,Jardim Fontalis,Bar,Grocery Store,Video Game Store,Zoo,Floating Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
7618,4.0,Jardim Vila Rica,Brazilian Restaurant,Gym / Fitness Center,Grocery Store,Chinese Restaurant,Middle Eastern Restaurant,Pharmacy,Candy Store,Farmers Market,Food Truck,Food Stand
9095,4.0,Casa Verde Alta,Grocery Store,Market,Bar,Chocolate Shop,Pharmacy,Gym / Fitness Center,Convenience Store,Outlet Store,Bakery,Department Store
9293,4.0,Vila Santa Maria,Grocery Store,Coffee Shop,Furniture / Home Store,Dessert Shop,Pizza Place,Big Box Store,Gym,Bar,Fishing Spot,Fishing Store
9382,4.0,Chácara Morro Alto,Grocery Store,Farmers Market,Japanese Restaurant,Bagel Shop,Pet Store,Bar,Gym,Big Box Store,Flea Market,Financial or Legal Service
10267,4.0,Jardim Primavera (Zona Norte),Grocery Store,Pizza Place,Gym,Dessert Shop,Bus Station,Big Box Store,Bar,Park,Bakery,BBQ Joint


In [165]:
c44.shape

(95, 12)

In [166]:
# Cluster 5
c5 = Tmerge.loc[Tmerge['Cluster Labels'] == 5, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c55 = c5[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c55.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5.0,Sé,Bookstore,Bakery,Jewelry Store,Historic Site,Snack Place,Miscellaneous Shop,Café,Brazilian Restaurant,Music Store,Arts & Crafts Store
9,5.0,Centro,Café,Brazilian Restaurant,Bookstore,Jewelry Store,Chocolate Shop,Bar,Bakery,Snack Place,Historic Site,Coffee Shop
173,5.0,Brás,Brazilian Restaurant,Clothing Store,Furniture / Home Store,Italian Restaurant,Shopping Mall,Hotel,Department Store,Burger Joint,Restaurant,Shoe Store
540,5.0,Luz,Brazilian Restaurant,Hotel,Noodle House,Bus Stop,Café,Snack Place,Mineiro Restaurant,Gym,Bakery,Farmers Market
546,5.0,Ponte Pequena,Sports Bar,Zoo,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna
591,5.0,Bom Retiro,Diner,Restaurant,Women's Store,Korean Restaurant,Baseball Stadium,Café,Pizza Place,Bar,Borek Place,Brazilian Restaurant
758,5.0,Parque Industrial Tomas Edson,Nightclub,Motel,Soccer Field,Brazilian Restaurant,Sporting Goods Shop,Theater,Supermarket,Gas Station,Italian Restaurant,Arts & Crafts Store
820,5.0,Santa Cecília,Pizza Place,Gym / Fitness Center,Bakery,Italian Restaurant,Pharmacy,Restaurant,Bookstore,Garden Center,Supermarket,Steakhouse
874,5.0,Perdizes,Dessert Shop,Pharmacy,Burger Joint,Bakery,Vegetarian / Vegan Restaurant,Italian Restaurant,Bar,Argentinian Restaurant,Brazilian Restaurant,Bookstore
916,5.0,Santa Efigênia,Brazilian Restaurant,Electronics Store,Hotel,Music Store,Plaza,Peruvian Restaurant,Concert Hall,Café,Bakery,Middle Eastern Restaurant


In [167]:
c55.shape

(681, 12)

In [177]:
c55

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5.0,Sé,Bookstore,Bakery,Jewelry Store,Historic Site,Snack Place,Miscellaneous Shop,Café,Brazilian Restaurant,Music Store,Arts & Crafts Store
9,5.0,Centro,Café,Brazilian Restaurant,Bookstore,Jewelry Store,Chocolate Shop,Bar,Bakery,Snack Place,Historic Site,Coffee Shop
173,5.0,Brás,Brazilian Restaurant,Clothing Store,Furniture / Home Store,Italian Restaurant,Shopping Mall,Hotel,Department Store,Burger Joint,Restaurant,Shoe Store
540,5.0,Luz,Brazilian Restaurant,Hotel,Noodle House,Bus Stop,Café,Snack Place,Mineiro Restaurant,Gym,Bakery,Farmers Market
546,5.0,Ponte Pequena,Sports Bar,Zoo,Flower Shop,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna
591,5.0,Bom Retiro,Diner,Restaurant,Women's Store,Korean Restaurant,Baseball Stadium,Café,Pizza Place,Bar,Borek Place,Brazilian Restaurant
758,5.0,Parque Industrial Tomas Edson,Nightclub,Motel,Soccer Field,Brazilian Restaurant,Sporting Goods Shop,Theater,Supermarket,Gas Station,Italian Restaurant,Arts & Crafts Store
820,5.0,Santa Cecília,Pizza Place,Gym / Fitness Center,Bakery,Italian Restaurant,Pharmacy,Restaurant,Bookstore,Garden Center,Supermarket,Steakhouse
874,5.0,Perdizes,Dessert Shop,Pharmacy,Burger Joint,Bakery,Vegetarian / Vegan Restaurant,Italian Restaurant,Bar,Argentinian Restaurant,Brazilian Restaurant,Bookstore
916,5.0,Santa Efigênia,Brazilian Restaurant,Electronics Store,Hotel,Music Store,Plaza,Peruvian Restaurant,Concert Hall,Café,Bakery,Middle Eastern Restaurant


In [168]:
# Cluster 6
c6 = Tmerge.loc[Tmerge['Cluster Labels'] == 6, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c66 = c6[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c66.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4283,6.0,Jardim Julieta,Bakery,Shopping Mall,Rest Area,Pizza Place,American Restaurant,Bus Station,Supermarket,Fast Food Restaurant,Fish & Chips Shop,Fish Market
4580,6.0,Vila Ede,Bakery,Gym / Fitness Center,Pizza Place,Food & Drink Shop,Zoo,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market
4701,6.0,Vila São Camilo,Bakery,Pizza Place,Gym / Fitness Center,Bus Stop,American Restaurant,Food,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market
5955,6.0,Guapira,Bakery,Northeastern Brazilian Restaurant,Middle Eastern Restaurant,Bar,Gym,Soccer Field,Zoo,Flea Market,Financial or Legal Service,Fireworks Store
6307,6.0,Vila Nova Mazzei,Bakery,Hardware Store,Burger Joint,Arts & Crafts Store,Pizza Place,Zoo,Flower Shop,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
6594,6.0,Vila Paulistana,Bakery,Supermarket,Bus Station,Fast Food Restaurant,Grocery Store,Zoo,Flower Shop,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
7269,6.0,Bortolândia,Bakery,Gym,Zoo,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna
7379,6.0,Jardim Santo Alberto,Bakery,Shopping Mall,Bus Station,Pharmacy,Gym / Fitness Center,Health & Beauty Service,Gas Station,Forest,Fish Taverna,Fast Food Restaurant
7628,6.0,Jardim Cruz do Corisco,Bakery,Record Shop,Convenience Store,Dog Run,Pizza Place,French Restaurant,Floating Market,Field,Financial or Legal Service,Fireworks Store
8840,6.0,Jardim Laranjeiras,Market,Bakery,Zoo,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna


In [169]:
c66.shape

(65, 12)

In [170]:
# Cluster 7
c7 = Tmerge.loc[Tmerge['Cluster Labels'] == 7, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c77 = c7[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c77.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
337,7.0,República,Coffee Shop,Brazilian Restaurant,Bar,Italian Restaurant,Theater,Restaurant,Café,Hotel,Cocktail Bar,Vegetarian / Vegan Restaurant
602,7.0,Canindé,Brazilian Restaurant,Middle Eastern Restaurant,Gift Shop,Restaurant,Pizza Place,South American Restaurant,Southeastern Brazilian Restaurant,Flower Shop,Shop & Service,Toy / Game Store
723,7.0,Barra Funda,Restaurant,Nightclub,Bar,Brazilian Restaurant,Martial Arts Dojo,Beer Bar,Plaza,Sandwich Place,Coffee Shop,Buffet
734,7.0,Várzea da Barra Funda,Brazilian Restaurant,Restaurant,Café,Sushi Restaurant,Japanese Restaurant,Sandwich Place,Music Venue,Bus Station,Bookstore,Nightclub
876,7.0,Campos Elíseos,Brazilian Restaurant,Peruvian Restaurant,Hotel,Middle Eastern Restaurant,Italian Restaurant,Bar,Cocktail Bar,Bakery,Pizza Place,Gay Bar
1027,7.0,Vila Buarque,Bar,Pizza Place,Brazilian Restaurant,Bakery,Restaurant,Theater,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Arts & Crafts Store
1095,7.0,Consolação,Bar,Brazilian Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Mineiro Restaurant,Gym / Fitness Center,Café,Coffee Shop,Italian Restaurant,Food Truck
1562,7.0,Bela Vista,Bar,Italian Restaurant,Theater,Bakery,Pizza Place,Café,Brazilian Restaurant,Gymnastics Gym,Diner,Optical Shop
1901,7.0,Morro dos Ingleses,Brazilian Restaurant,Restaurant,Nightclub,Bar,Café,Pizza Place,Japanese Restaurant,Hotel,Hostel,Gymnastics Gym
2526,7.0,Itaim Bibi,Burger Joint,Restaurant,Bar,Brazilian Restaurant,Japanese Restaurant,Coffee Shop,Middle Eastern Restaurant,Gym,Italian Restaurant,Steakhouse


In [171]:
c77.shape

(278, 12)

In [172]:
# Cluster 8
c8 = Tmerge.loc[Tmerge['Cluster Labels'] == 8, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c88 = c8[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c88.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3679,8.0,Vila Isolina Mazzei,Farmers Market,Brewery,Food Truck,Gym,Veterinarian,Floating Market,Fast Food Restaurant,Field,Financial or Legal Service,Fireworks Store
5588,8.0,Vila Pedrosa,Pizza Place,Convenience Store,Japanese Restaurant,Electronics Store,Bakery,Food Truck,Mobile Phone Shop,Flea Market,Fishing Store,Fishing Spot
6134,8.0,Conjunto Habitacional Jova Rural,Health & Beauty Service,Fast Food Restaurant,Food Truck,Plaza,Zoo,Flea Market,Farmers Market,Field,Financial or Legal Service,Fireworks Store
6596,8.0,Jardim Joamar,Pizza Place,Pharmacy,Grocery Store,Other Great Outdoors,Gym,Food Truck,Fast Food Restaurant,Women's Store,Fireworks Store,Fish & Chips Shop
6741,8.0,Jardim Martins Silva,Bakery,Food Truck,Food & Drink Shop,Grocery Store,Zoo,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
6824,8.0,Jardim Filhos da Terra,Fast Food Restaurant,Health & Beauty Service,Burger Joint,Food Truck,Moving Target,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
6917,8.0,Jardim Portal I e II,Food Truck,Grocery Store,Burger Joint,Bar,Video Game Store,Zoo,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop
6964,8.0,Jardim Hebrom,Food Truck,Shopping Mall,Health & Beauty Service,Boutique,Pizza Place,Bakery,Moving Target,Fish Market,Fishing Spot,Fish Taverna
7419,8.0,Jardim Daysy,Steakhouse,Bagel Shop,Plaza,Food Truck,Flower Shop,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market
7492,8.0,Jardim Flor de Maio,Restaurant,Food Truck,Video Game Store,Grocery Store,Zoo,Flea Market,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service


In [173]:
c88.shape

(96, 12)

In [174]:
# Cluster 9
c9 = Tmerge.loc[Tmerge['Cluster Labels'] == 9, Tmerge.columns[[2] + list(range(3, Tmerge.shape[1]))]]
c99 = c9[['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', 
          '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 
          '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', 
          '9th Most Common Venue', '10th Most Common Venue']]
c99.head(10)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7669,9.0,Núcleo do Engordador,Campground,Zoo,Farmers Market,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Spot
36639,9.0,Jardim das Palmeiras (Zona Sul),Campground,Zoo,Farmers Market,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Spot
41391,9.0,Jardim Paineiras,Campground,Zoo,Farmers Market,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Spot
48453,9.0,Jardim das Palmeiras,Campground,Zoo,Farmers Market,Field,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Spot


In [175]:
c99.shape

(4, 12)